In [103]:
from scipy.optimize import fsolve
import numpy as np

In [118]:
def reynoldsNumber(density, velocity, diameter, viscosity) -> float:
    """
    Return the reynold's number
    """
    den = density
    vel = velocity
    dia = diameter
    vis = viscosity
    print("velocity ", vel)
    reynolds_number = (den*vel*dia)/vis
    # print("Reynold's number ", reynolds_number)
    return reynolds_number

To calculate the friction factor, we need to take into account if it is a turbulent or laminar flow. 

For turbulent flows, I am using an initial guess of the complete turbulence value in order to use the Scipy method "fsolve".

In [115]:
def frictionFactorCompleteTurbulence(pipe_roughness, diameter):
    """
    Return the friction factor for complete turbulence. We use this to guess our initial value for turbulent flows
    """
    pip = pipe_roughness
    dia = diameter

    return (-2*np.log((pip/dia)/3.7))**(-2)

def frictionFactor(reynolds_number, pipe_roughness, diameter):
    """
    Return the friction factor
    Determine if friction factor is turbulent or laminar
    """
    rey = reynolds_number
    pip = pipe_roughness
    dia = diameter
    # print("reynoldsNumber ", reynolds_number)
    # if turbulent flow
    if (reynolds_number > 4000):
        f_complete = frictionFactorCompleteTurbulence(pip, dia)
        def calculateFrictionFactor(x):
            return (1/x**0.5) + 2*np.log10(((pip/dia)/3.7) + (2.51/(rey*x**0.5)))

        # Solve the equation
        # plug in the initial guess in second paramter
        solution = fsolve(calculateFrictionFactor, f_complete)
        # print("Friction factor ", solution[0])
        return solution[0]
    # else return laminar flow friction factor
    return 64/rey

The equation we got for V_2 is the following:
$z_1 +\frac{p_{\textrm{atm}} }{\rho\,g}+\frac{\alpha\,{A_2 }^2 \,V_2^2 }{2\,{A_1 }^2 \,g}=z_2 +\frac{k\,V_2^2 }{2g\ }+\frac{P_1+\rho\,g\,h}{\rho\,g}+\frac{\alpha\,V_2^2 }{2\,g}+\frac{F\,l\,V_2^2 }{2\,D\,g}$

In [142]:
def getV2(z_2, z_1, friction_factor, length, diameter, p_2, v2_guess):
    """
    Calculate V_2
    """
    # set constants
    g = 9.81
    alpha = 1
    k = 0.5
    p_atm = 101.325
    roh = 1000

    # set parameters
    fri = friction_factor
    dia = diameter
    h = z_1 # height
    
    a_2 = (np.pi * (diameter/2)**2) # area of pipe
    a_1 = 0.32 * 0.26 # area of block
    # let x = v_2
    def calculateV2(x):
        (p_atm/roh*g) + ((alpha*((a_2/a_1) * x)**2)/(2*g)) + z_1 - ((p_2 + (roh * g * h))/(roh * g)) - ((alpha * x ** 2)/(2*g)) - z_2 - (fri * (length / dia) * ((x**2)/2*g)) - ((k*x**2)/(2*g))
    
    solution = fsolve(calculateV2, v2_guess)
    # print("Calculated v_2 ", solution[0])
    return solution[0]

In [107]:
# Our relation of V_1 to V_2
def calculateV1(v_2):
    return (5.9512*10**-4) * v_2

In the following, we use fixed point iteration to calculate the correct V_2 value (when v2_guess = calculated_v2).

In [141]:
def calculateV2(tube_length, height, v2_guess):
    # set constants
    tube_diameter = 0.00794 #m
    density_water = 1000
    viscosity = 0.0010005
    pipe_roughness = 1.5*10**-6 # m
    z_1 = height
    p_atm = 101.325
    p_2 = p_atm

    # set variables
    # z_2 is start height of water
    z_2 = 0

    # print("z_1 ", z_1)


    # calculate reynold's number
    reynolds_number = reynoldsNumber(density_water, v2_guess, tube_diameter, viscosity)

    # calculate friction factor
    friction_factor = frictionFactor(reynolds_number, pipe_roughness, tube_diameter)

    # calculate the new v_2
    # z_1 changes each time as it's equal to h
    calculated_v2 = getV2(z_2, z_1, friction_factor, tube_length, tube_diameter, p_2, v2_guess)

    # initial tolerance
    tol = 10

    # fixed point iteration
    while tol>0.000001:
        # print("\n")
        # print("v2_guess ", v2_guess, "calculated_v2 ", calculated_v2)
        v2_guess = calculated_v2
        reynolds_number = reynoldsNumber(density_water, v2_guess, tube_diameter, viscosity)
        friction_factor = frictionFactor(reynolds_number, pipe_roughness, tube_diameter)
        calculated_v2 = getV2(z_2, z_1, friction_factor, tube_length, tube_diameter, p_2, v2_guess)
        tol = abs(v2_guess - calculated_v2)

    # print("final v2 ", calculated_v2, "guess ", v2_guess, "tol ", tol)
    return calculated_v2



This is the starting block of our functions.
We set the $\Delta$ T and then we keep on calculating T until h is less than 0.
Our total time is summed up each iteration.

In [143]:
# set parameters
tube_length = 0.20
z_1 = 0.08 + 0.02 + (1/150 * tube_length)
v2_guess = 10
h = z_1
delta_t = 1 # delta_t = 1 second
sum_time = 0

# v_2 = calculateV2(tube_length)
# v_1 = calculateV1(v_2)
# print("v_1 ", v_1, "v_2 ", v_2)

while h > 0:
    v_2 = calculateV2(tube_length, h, v2_guess)
    v_1 = calculateV1(v_2)
    # print("h ", h)
    # print("v_1 ", v_1)
    h = h - (delta_t)*v_1
    sum_time += delta_t

print("h ", h)
print("sum_time ", sum_time)

velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984007994
velocity  10.0
reynoldsNumber  79360.31984007994
velocity  10
reynoldsNumber  79360.31984

/var/folders/80/sf5zrh2x5p9c8h3qmrspk3h40000gn/T/ipykernel_22346/2760412591.py:23: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  solution = fsolve(calculateV2, v2_guess)
